## Load Llama-3-8B

In [1]:
# Warning: Using transformer version in DPO will lead to errors of loading Llama3
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer
transformers.logging.set_verbosity_error()
from trl import setup_chat_format

[2024-07-16 15:50:59,963] [INFO] [real_accelerator.py:203:get_accelerator] Setting ds_accelerator to cuda (auto detect)
 [WARNING]  async_io requires the dev libaio .so object and headers but these were not found.


/opt/conda/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


 [WARNING]  async_io: please install the libaio-dev package with apt
 [WARNING]  If libaio is already installed (perhaps from source), try setting the CFLAGS and LDFLAGS environment variables to where it can be found.
 [WARNING]  Please specify the CUTLASS repo directory as environment variable $CUTLASS_PATH
 [WARNING]  sparse_attn requires a torch version >= 1.5 and < 2.0 but detected 2.3
 [WARNING]  using untested triton version (2.3.0), only 1.0.0 is known to be compatible


In [76]:
tokenizer = AutoTokenizer.from_pretrained("<some path>/model_zoo/Meta-Llama-3-8B-Instruct",device_map="auto")
model = AutoModelForCausalLM.from_pretrained("<some path>/model_zoo/Meta-Llama-3-8B-Instruct",device_map="auto")

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [77]:
tokenizer.chat_template

"{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}"

# Datasets

In [79]:
from datasets import load_dataset,DatasetDict
#hh_dataset = load_dataset("Anthropic/hh-rlhf")
ds = load_dataset("<some path>/HuggingFaceH4_ultrafeedback_binarized")

In [80]:
ds

DatasetDict({
    train_prefs: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 61135
    })
    train_sft: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 61135
    })
    test_prefs: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 2000
    })
    test_sft: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 1000
    })
    train_gen: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 61135
    })
    test_gen: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows

In [82]:
ds_rlhf = DatasetDict()

In [83]:
for split in ["train","test"]:   
    ds_rlhf[split] = ds[split+"_prefs"]

In [84]:
ds_rlhf

DatasetDict({
    train: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 61135
    })
    test: Dataset({
        features: ['prompt', 'prompt_id', 'chosen', 'rejected', 'messages', 'score_chosen', 'score_rejected'],
        num_rows: 2000
    })
})

In [85]:
ds_rlhf=ds_rlhf.remove_columns(["prompt_id",'messages', 'score_chosen', 'score_rejected'])

In [87]:
ds_rlhf["train"]["chosen"][0]

[{'content': 'how can i develop a habit of drawing daily', 'role': 'user'},
 {'content': "Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help build the habit.\n3. Start small and simple: Don't try to create a masterpiece every day, start with simple and easy-to-do sketches. Focus on improving your skills gradually.\n4. Use a variety of tools and mediums: Experiment with different tools like pencils, pens, markers, and different mediums like paper

In [88]:
ds_rlhf.save_to_disk("<some path>/trl/examples/datasets/UltraFeedback_trl/")

Saving the dataset (0/1 shards):   0%|          | 0/61135 [00:00<?, ? examples/s]

Saving the dataset (0/1 shards):   0%|          | 0/2000 [00:00<?, ? examples/s]

In [14]:
from typing import Optional, Literal

LLaMa3_CHAT_template = "{% set loop_messages = messages %}{% for message in loop_messages %}{% set content = '<|start_header_id|>' + message['role'] + '<|end_header_id|>\n\n'+ message['content'] | trim + '<|eot_id|>' %}{% if loop.index0 == 0 %}{% set content = bos_token + content %}{% endif %}{{ content }}{% endfor %}{% if add_generation_prompt %}{{ '<|start_header_id|>assistant<|end_header_id|>\n\n' }}{% endif %}",


## DataSet Checking

In [2]:
from datasets import load_from_disk
dataset_path = "<some path>/trl/examples/datasets/UltraChat_sft_trl"
UltraChat_sft_trl = load_from_disk(dataset_path)
UltraChat_sft_trl["train"][0]

{'messages': [{'content': "These instructions apply to section-based themes (Responsive 6.0+, Retina 4.0+, Parallax 3.0+ Turbo 2.0+, Mobilia 5.0+). What theme version am I using?\nOn your Collections pages & Featured Collections sections, you can easily show the secondary image of a product on hover by enabling one of the theme's built-in settings!\nYour Collection pages & Featured Collections sections will now display the secondary product image just by hovering over that product image thumbnail.\nDoes this feature apply to all sections of the theme or just specific ones as listed in the text material?",
   'role': 'user'},
  {'content': 'This feature only applies to Collection pages and Featured Collections sections of the section-based themes listed in the text material.',
   'role': 'assistant'},
  {'content': 'Can you guide me through the process of enabling the secondary image hover feature on my Collection pages and Featured Collections sections?',
   'role': 'user'},
  {'conten

In [ ]:
UltraChat_sft_trl["train"][0]

In [89]:
from datasets import load_from_disk
dataset_path = "<some path>/trl/examples/datasets/UltraFeedback_trl"
UltraFeedback_trl = load_from_disk(dataset_path)
UltraFeedback_trl["train"][0]

{'prompt': 'how can i develop a habit of drawing daily',
 'chosen': [{'content': 'how can i develop a habit of drawing daily',
   'role': 'user'},
  {'content': "Developing a daily habit of drawing can be challenging but with consistent practice and a few tips, it can become an enjoyable and rewarding part of your daily routine. Here are some strategies to help you develop the habit of drawing daily:\n\n1. Set a specific time: Allocate a specific time of the day to draw. It could be in the morning, afternoon, or evening. Make drawing a part of your daily routine.\n2. Set a specific duration: Determine the amount of time you want to spend on drawing each day. It can be as little as 10 minutes or as long as an hour. Be consistent with the duration to help build the habit.\n3. Start small and simple: Don't try to create a masterpiece every day, start with simple and easy-to-do sketches. Focus on improving your skills gradually.\n4. Use a variety of tools and mediums: Experiment with diffe

# SFT

## SFT: Llama 3

In [ ]:
# FSDP
!accelerate launch --config_file=examples/accelerate_configs/FSDP_Llama.yaml --num_processes 8 examples/scripts/sft_fsdp.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraChat_sft_trl" \
    --model_name_or_path="<some path>/model_zoo/Meta-Llama-3-8B" \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --attn_implementation 'flash_attention_2' \
    --learning_rate 6e-7 \
    --gradient_accumulation_steps 8 \
    --gradient_checkpointing \
    --logging_steps 10 \
    --output_dir="models/Llama3_sft_UltraChat" \
    --optim adamw_torch \
    --seed 42 \
    --save_steps 5000\
    --warmup_steps 150 #\

In [ ]:
# FSDP (SimPO hyperparameters)
!accelerate launch --config_file=examples/accelerate_configs/FSDP_Llama.yaml --num_processes 8 examples/scripts/sft_fsdp.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraChat_sft_trl" \
    --model_name_or_path="<some path>/model_zoo/Meta-Llama-3-8B" \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --attn_implementation 'flash_attention_2' \
    --learning_rate 2e-5 \
    --gradient_accumulation_steps 8 \
    --gradient_checkpointing \
    --logging_steps 10 \
    --output_dir="models/Llama3_sft-UltraChat_lr_2e-5" \
    --optim adamw_torch \
    --seed 42 \
    --save_steps 5000\
    --warmup_steps 150 \
    --max_seq_length 2048


In [ ]:
# Deepspeed Zero3 (SimPO hyperparameters)
!accelerate launch --config_file=examples/accelerate_configs/deepspeed_zero3_sft.yaml --num_processes 8 examples/scripts/sft_zero3.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraChat_sft_trl" \
    --model_name_or_path="<some path>/model_zoo/Meta-Llama-3-8B" \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 1 \
    --attn_implementation 'flash_attention_2' \
    --learning_rate 2e-5 \
    --gradient_accumulation_steps 16 \
    --gradient_checkpointing \
    --output_dir="models/Llama3_sft-UltraChat_SimPOhyper" \
    --optim adamw_torch \
    --seed 42 \
    --save_steps 5000\
    --warmup_ratio 0.1 \
    --max_seq_length 2048 \
    --bf16 \
    --logging_steps 10 \

## SFT: Llama3-Instruct (Is this one needed?)

In [ ]:
# FSDP Llama3-Instruct
!accelerate launch --config_file=examples/accelerate_configs/FSDP_Llama.yaml --num_processes 8 examples/scripts/sft_fsdp.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraChat_sft_trl" \
    --model_name_or_path="<some path>/model_zoo/Meta-Llama-3-8B-Instruct" \
    --attn_implementation 'flash_attention_2' \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --learning_rate 6e-7 \
    --gradient_accumulation_steps 8 \
    --gradient_checkpointing \
    --logging_steps 10 \
    --output_dir="Llama3-Instruct_sft_UltraChat" \
    --optim adamw_torch \
    --seed 42 \
    --save_steps 5000\
    --warmup_steps 150 \

In [ ]:
# FSDP Llama3-Instruct (SimPO hyperparameters)
!accelerate launch --config_file=examples/accelerate_configs/FSDP_Llama.yaml --num_processes 8 examples/scripts/sft_fsdp.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraChat_sft_trl" \
    --model_name_or_path="<some path>/model_zoo/Meta-Llama-3-8B-Instruct" \
    --attn_implementation 'flash_attention_2' \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 1 \
    --learning_rate 2e-5 \
    --gradient_accumulation_steps 8 \
    --gradient_checkpointing \
    --logging_steps 10 \
    --output_dir="models/Llama3-Instruct_sft_UltraChat_lr_2e-5" \
    --optim adamw_torch \
    --seed 42 \
    --save_steps 5000\
    --warmup_steps 150 \
    --max_seq_length 2048

# RLHF (Deepspeed Zero)

## RLHF on Llama3 SFT

In [ ]:
# DPO on Llama3 sft 
!accelerate launch --config_file=examples/accelerate_configs/deepspeed_zero3.yaml --num_processes 8 examples/scripts/dpo_zero3.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraFeedback_trl" \
    --model_name_or_path="<some path>/trl/models_sft/Llama3_sft-UltraChat_SimPOhyper/checkpoint-4872" \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 2 \
    --attn_implementation 'flash_attention_2' \
    --learning_rate 1.0e-6 \
    --gradient_accumulation_steps 16 \
    --gradient_checkpointing True \
    --logging_steps 10 \
    --save_steps 50000 \
    --output_dir="<some path>/trl/models_rlhf/Llama3_sft-UltraChat_dpo" \
    --optim rmsprop \
    --max_length 2048 \
    --max_prompt_length 1800 \
    --warmup_steps 150 \
    --bf16 \
    --logging_first_step \
    --no_remove_unused_columns

In [ ]:
# IPO on Llama3 sft
!accelerate launch --config_file=examples/accelerate_configs/deepspeed_zero3.yaml --num_processes 8 examples/scripts/dpo_zero3.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraFeedback_trl" \
    --model_name_or_path="<some path>/trl/models_sft/Llama3_sft_UltraChat_SimPOhyper/checkpoint-4872" \
    --loss_type="ipo" \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 2 \
    --attn_implementation 'flash_attention_2' \
    --learning_rate 1.0e-6 \
    --gradient_accumulation_steps 16 \
    --gradient_checkpointing True \
    --logging_steps 10 \
    --save_steps 50000 \
    --output_dir="<some path>/trl/models_rlhf/Llama3_sft-UltraChat_ipo" \
    --optim rmsprop \
    --max_length 2048 \
    --max_prompt_length 1800 \
    --warmup_steps 150 \
    --bf16 \
    --logging_first_step \
    --no_remove_unused_columns

## RLHF on Llama3-Instruct

In [ ]:
# DPO on Llama-3-Instruct
!accelerate launch --config_file=examples/accelerate_configs/deepspeed_zero3.yaml --num_processes 8 examples/scripts/dpo_zero3.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraFeedback_trl" \
    --model_name_or_path="<some path>/model_zoo/Meta-Llama-3-8B-Instruct" \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 4 \
    --attn_implementation 'flash_attention_2' \
    --learning_rate 1.0e-6 \
    --gradient_accumulation_steps 8 \
    --gradient_checkpointing True \
    --logging_steps 10 \
    --save_steps 50000 \
    --output_dir="<some path>/trl/models_rlhf/Llama3-Instruct_dpo" \
    --optim rmsprop \
    --max_length 1024 \
    --max_prompt_length 512 \
    --warmup_steps 150 \
    --bf16 \
    --logging_first_step \
    --no_remove_unused_columns

In [ ]:
# DPO on Llama-3-Instruct (with same setting as SimPO)
!accelerate launch --config_file=examples/accelerate_configs/deepspeed_zero3.yaml --num_processes 8 examples/scripts/dpo_zero3.py \
    --dataset_name="<some path>/trl/examples/datasets/UltraFeedback_trl" \
    --model_name_or_path="<some path>/model_zoo/Meta-Llama-3-8B-Instruct" \
    --per_device_train_batch_size 1 \
    --per_device_eval_batch_size 2 \
    --learning_rate 1.0e-6 \
    --gradient_accumulation_steps 16 \
    --gradient_checkpointing True \
    --logging_steps 10 \
    --save_steps 50000 \
    --output_dir="<some path>/trl/models_rlhf/Llama3-Instruct_dpo_2048" \
    --optim rmsprop \
    --max_length 2048 \
    --max_prompt_length 1800 \
    --warmup_steps 150 \
    --bf16 \
    --logging_first_step \
    --no_remove_unused_columns \
    --attn_implementation 'flash_attention_2'